In [1]:
import pandas as pd

from loaders import load_geojson
from params import DATA_DIR

Load all the required types into a single `GeoDataFrame`.

In [2]:
data = pd.concat([
    load_geojson(type)
    for type
    in ['panrgn22', 'rgn22', 'cauth22', 'cty22', 'lad22', 'wd22'
]])

Check for and remove duplicates

In [3]:
data[data.index.duplicated()]

,RGN21CD,RGN21NM,geometry,ons_code,ons_type,RGN22CD,RGN22NM,CAUTH22CD,CAUTH22NM,CTY22CD,CTY22NM,LAD22CD,LAD22NM,WD22CD,WD22NM
ons_code,,,,,,,,,,,,,,,
E05014284,NaN,NaN,"MULTIPOLYGON (((-0.41824 54.17405, -0.41809 54...",E05014284,WD22CD,NaN,NaN,NaN,NaN,NaN,NaN,E07000167,Ryedale,E05014284,Hunmanby & Sherburn (part Scarborough)


In [4]:
data = data[~data.index.duplicated()]

Save to a parquet file for later processing. Could have written to a geojson file as follows:

```py
with open('places.geojson', 'w') as f:
    f.write(data[['ons_code', 'ons_type', 'geometry']].to_json())
```

In [21]:
data[['ons_code', 'ons_type', 'geometry']].to_parquet(f'{DATA_DIR}/interim/shapes.parquet')